In [1]:
import json

In [2]:
from collections import Counter

In [3]:
import pandas as pd

In [4]:
with open('main.json', encoding='utf-8') as fh:
    data = json.load(fh)


In [48]:
with open('tables_with_attributes.txt', encoding='utf-8') as fx:
    data2 = json.load(fx)


In [ ]:
def check_lists(list1,list2):
    return list(set(list1).intersection(list2))
    

In [41]:
#defining columns
cols =["table","condition","field","operator","value"]
#creating main dataframe for back-end
main_frame = pd.DataFrame(columns=cols)

#loop through all the main-keys(tables) from the JSON recieved

for each_key in list(data.keys()):
    
    #sample is variable which holds the value from the given key 
    #updated as loop goes through the JSON tree
    sample = data[each_key]
    
    #loop for travelling through tree structure/ tunneling down
    while 1>0:
        
        #checks if the sample has list of rules or not
        if len(sample['rules'])>0:
            
            #checks if key named "rules" available or not in the sample
            if 'rules' in list(sample.keys()):
                 
                #loop for interation on list of rules, except the last value as it is for new dictionary within JSON(branching)    
                for each in sample['rules'][:-1]:
                    
                    #checks if key named "value" available or not in the sample
                    if "value" in list(each.keys()):
                        
                        #this temp dataframe is created as one row to append to the main dataframe
                        temp = pd.DataFrame([[str(each_key),
                                              str(sample['condition']),
                                              str(each['field']),
                                              str(each['operator']),
                                              str(each['value'])
                                             ]],
                                            columns=cols)

                        #appending temp dataframe to main dataframe
                        main_frame = main_frame.append(temp,ignore_index=True,verify_integrity=True)

                
                #updating current sample to new sample/ tunneling down
                sample = list(sample['rules'])[-1]
        
        #if current sample does not have list of rules it breaks the tunneling loop
        else:
            #adding "" to the values
            main_frame['value'] = "\"" + main_frame['value'] + "\""
            #converting all the conditions to uppercase
            main_frame['condition'] = main_frame['condition'].str.upper()
            break
                        
#main_frame

In [38]:
data2

#defining columns
cols = ['constraint_catalog', 'constraint_schema', 'constraint_name', 'table_catalog', 'table_schema', 'table_name', 'constraint_type', 'is_deferrable', 'initially_deferred', 'enforced']
#creating main dataframe for back-end
temp_frame = pd.DataFrame(columns=cols)
join_frame = pd.DataFrame(index= list(main_frame['table'].drop_duplicates()),columns=list(main_frame['table'].drop_duplicates()))

#loop through all the main-keys(tables) from the JSON recieved

for each in list(data2):
        
                        
    #this temp dataframe is created as one row to append to the main dataframe
    temp = pd.DataFrame([[str(each['constraint_catalog']),
                        str(each['constraint_schema']),
                        str(each['constraint_name']),
                        str(each['table_catalog']),
                        str(each['table_schema']),
                        str(each['table_name']),
                        str(each['constraint_type']),
                        str(each['is_deferrable']),
                        str(each['initially_deferred']),
                        str(each['enforced']),                                   
                        ]],columns=cols)

    #appending temp dataframe to main dataframe
    temp_frame = temp_frame.append(temp,ignore_index=True,verify_integrity=True)

#removing rows where constraint_type = CHECK
temp_frame = temp_frame[['constraint_name','table_name','constraint_type']]
#temp_frame

In [56]:
cols = ['table','columns']

#creating table info. dataframe for back-end
table_frame = pd.DataFrame(columns=cols)

#loop through all the main-keys(tables) from the JSON recieved

for each in list(data2):     
                        
    #this temp dataframe is created as one row to append to the main dataframe
    temp = pd.DataFrame([[str(each['table']),
                        each['columns']]],columns=cols)

    #appending temp dataframe to main dataframe
    table_frame = table_frame.append(temp,ignore_index=True,verify_integrity=True)

#removing rows where constraint_type = CHECK
#temp_frame = temp_frame[['constraint_name','table_name','constraint_type']]
table_frame

,table,columns
0,account,"[account_rollup, account_type, account_id, acc..."
1,category,"[category_parent, category_rollup, category_de..."
2,currency,"[currency_id, conversion_ratio, currency, date]"
3,customer,"[city, customer_id, account_num, lname, fname,..."
4,days,"[week_day, day]"
5,department,"[department_description, department_id]"
6,employee,"[salary, first_name, supervisor_id, employee_i..."
7,employee_closure,"[supervisor_id, distance, employee_id]"
8,expense_fact,"[currency_id, account_id, exp_date, time_id, c..."
9,inventory_fact,"[product_id, warehouse_id, store_id, units_ord..."


In [26]:
for each in (main_frame['table'].drop_duplicates()):
    print(each,": ...")
    print(temp_frame[temp_frame.table_name==each])

account : ...
  constraint_catalog constraint_schema constraint_name table_catalog  \
0           foodmart            public      pk_account      foodmart   

  table_schema table_name constraint_type is_deferrable initially_deferred  \
0       public    account     PRIMARY KEY            NO                 NO   

  enforced  
0      YES  
customer : ...
   constraint_catalog constraint_schema constraint_name table_catalog  \
3            foodmart            public     pk_customer      foodmart   
36           foodmart            public     region14260      foodmart   

   table_schema table_name constraint_type is_deferrable initially_deferred  \
3        public   customer     PRIMARY KEY            NO                 NO   
36       public   customer     FOREIGN KEY            NO                 NO   

   enforced  
3       YES  
36      YES  


In [22]:
table1= "account"
table2= "expense_fact"
def if_join_or_not(table1,table2):
    for each in list(temp_frame['constraint_name']):
        if table1 in str(each):
            print(each)
            if each in temp_frame[temp_frame['table_name']==table2]:
                print("yes")

print(if_join_or_not(table1,table2))

pk_account
account1900
None


In [23]:
temp_frame[temp_frame['constraint_name']=='account1900']['table_name']=='expense_fact'

20    True
Name: table_name, dtype: bool

In [12]:
#Labelling the tables
#defining columns
cols =["table","reference"]
#creating labelld tables list for back-end
labelld_tables = pd.DataFrame(columns=cols)

#copying table names from main_frame and removing the duplicates and reseting index
labelld_tables["table"] = main_frame['table'].drop_duplicates()
labelld_tables.reset_index(drop=True,inplace=True)

#asigning each table an alphabate 
for each_table in list(labelld_tables['table'].index):
    
    if each_table>122:
        #write an error message here
        break
    labelld_tables['reference'].iloc[each_table] = chr(each_table+97)

    
labelld_tables

,table,reference
0,account,a
1,customer,b


In [13]:
main_frame = pd.merge(main_frame,labelld_tables,how='left', on=['table'])
main_frame

,table,condition,field,operator,value,reference
0,account,AND,birthday,=,"""""2020-04-06T12:23:29.201Z""""",a
1,account,OR,gender,=,"""""male""""",a
2,account,OR,notes,=,"""""false""""",a
3,customer,AND,birthday,=,"""2020-04-06T12:23:29.201Z""",b


In [14]:
# checking if there is only one table available for query
if len(data.keys())<=1:
    
    #this is a part of the query, it contains name of tables separated by coma
    col_list = ""

    #list of keywords which can be used in query
    keywords = ["SELECT "," FROM "," WHERE "]
    
    #starting the query with "select"
    query = keywords[0]
    
    #loop to append column names to col_list
    for column in main_frame['field']:
        col_list = col_list + str(column) + ","

    #removing the last coma from the col_list
    col_list = col_list[:-1]
    
    #adding other kwywords and table name
    query = query + col_list + keywords[1] + str(list(main_frame["table"])[0]) + keywords[2]
    
    #loop to append conditions
    for row_number in list(main_frame.index):
        
        if row_number==0:
            for each in list(main_frame[['field','operator','value']].iloc[row_number]):
                query = query + " "+ each
        else:
            for each in list(main_frame[['condition','field','operator','value']].iloc[row_number]):
                query = query + " "+ each

    #ending query with ":"
    query = query+ ";"

    print("Generated Query:"+"\n"+query)
    
    
    
    
    
    
# query with more than one table
else :
    
    #this is a part of the query, it contains name of tables separated by coma
    col_list = ""

    #list of keywords which can be used in query
    keywords = ["SELECT "," FROM "," WHERE "]
    
    #starting the query with "select"
    query = keywords[0]
    
    #loop to append column names to col_list
    for row_number in list(main_frame.index):
        col_list = col_list + main_frame.iloc[row_number]['reference'] + '.' + main_frame.iloc[row_number]['field'] + ','

    #removing the last coma from the col_list
    col_list = col_list[:-1]
    
    #adding other kwywords and table name
    query = query +' '+ col_list + keywords[1] + str(list(main_frame["table"])[0]) + " "+ str(list(main_frame["reference"])[0]) + keywords[2]
    
    
    #loop for writing joins
    
    
    
    #loop to append conditions
    for row_number in list(main_frame.index):
        
        #query = query + main_frame['reference'].iloc[row_number] +'.'+main_frame['field'].iloc[row_number]
        
        if row_number==0:
            query = query + main_frame['reference'].iloc[row_number] +'.'+main_frame['field'].iloc[row_number]
            for each in list(main_frame[['operator','value']].iloc[row_number]):           
                query = query + " "+ each
        else:
            query = query + ' '+main_frame['condition'].iloc[row_number]+" "+ main_frame['reference'].iloc[row_number] +'.'+main_frame['field'].iloc[row_number]
            for each in list(main_frame[['operator','value']].iloc[row_number]):
#                query = query + main_frame['reference'].iloc[row_number] +'.'+main_frame['field'].iloc[row_number]        
                query = query + " "+ each

    #ending query with ":"
    query = query+ ";"

    print("Generated Query:"+"\n"+query)

Generated Query:
SELECT  a.birthday,a.gender,a.notes,b.birthday FROM account a WHERE a.birthday = ""2020-04-06T12:23:29.201Z"" OR a.gender = ""male"" OR a.notes = ""false"" AND b.birthday = "2020-04-06T12:23:29.201Z";
